# Object References, Mutability, and Recycling

## Variables Are Not Boxes

In python: variables are labels, not boxes. 

In [1]:
# . Variables are assigned to objects only after the objects are created

class Gizmo:
    def __init__(self) -> None:
        print(f'Gizmo id: {id(self)}')

In [2]:
x = Gizmo()

Gizmo id: 2538552574160


In [3]:
y = Gizmo() * 10

Gizmo id: 2538552575216


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [5]:
y

NameError: name 'y' is not defined

1. The output Gizmo id: ... is a side effect of creating a Gizmo instance.
2. Multiplying a Gizmo instance will raise an exception
3. Here is proof that a second Gizmo was actually instantiated before the multiplication was attempted.
4. But variable y was never created, because the exception happened while the right hand side of the assignment was being evaluated.

- To understand an assignment in Python, always read the righthand side first: that’s where the object is created or retrieved. After that, the variable on the left is bound to the object, like a label stuck to it

## Identity, Equality, Aliases

Because variables are mere labels, nothing prevents an object from having several labels assigned to it.

In [6]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [7]:
id(charles), id(lewis)

(2538564925120, 2538564925120)

In [8]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

1. lewis is an alias for charles.
2. The is operator and the id function confirm it.
3. Adding an item to lewis is the same as adding an item to charles.

In [9]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles

True

In [10]:
alex is not charles

True

1. alex refers to an object that is a replica of the object assigned to charles
2. The objects compare equal, because of the `__eq__` implementation in the dict class.
3. But they are distinct objects. This is the Pythonic way of writing the negative identity comparison: a is not b

Every object has an identity, a type and a value. An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The is operator compares the identity of two objects; the id() function returns an integer representing its identity

### Choosing Between == and is

The == operator compares the `values` of objects (the data they hold), while is compares their `identities`

### The Relative Immutability of Tuples

Tuples, like most Python collections—lists, dicts, sets, etc.—hold references to objects.If the referenced items are mutable, they `may change even if the tuple itself does no`t. In other words, the immutability of tuples really refers to the physical contents of the
`tuple data structure (i.e., the references it holds), and does not extend to the referenced objects`.

In [11]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

t1 == t2

True

In [12]:
id(t1[-1])

2538565644032

In [13]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [14]:
id(t1[-1])

2538565644032

1. t1 is immutable, but t1[-1] is mutable.
2. Inspect the identity of the list at t1[-1].
3. Modify the t1[-1] list in place.
4. The identity of t1[-1] has not changed, only its value.

## Copies Are Shallow by Default

In [16]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [18]:
l2 is l1

False

1. list(l1) creates a copy of l1
2. The copies are equal
3. But refer to two different objects

copy is filled with references to the same items held by
the original container

In [19]:
l1.append(100)
l1[1].remove(55) 
print('l1:', l1)
print('l2:', l2)

l1: [3, [44], (7, 8, 9), 100]
l2: [3, [44], (7, 8, 9)]


In [20]:
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 33, 22], (7, 8, 9), 100]
l2: [3, [44, 33, 22], (7, 8, 9, 10, 11)]


1. Appending 100 to l1 has no effect on l2
2. Here we remove 55 from the inner list l1[1]. This affects l2 because l2[1] is bound to the same list as l1[1].
3. For a mutable object like the list referred by l2[1], the operator += changes the list in place. This change is visible at l1[1], which is an alias for l2[1].
4. += on a tuple creates a new tuple and rebinds the variable l2[2] here.Now the tuples in the last position of l1 and l2 are no longer the same object.

### Deep and Shallow Copies of Arbitrary Objects

In [23]:
class Bus:
    def __init__(self, passengers = None) -> None:
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name: str):
        self.passengers.append(name)
        
    def drop(self, name: str):
        self.passengers.remove(name)

In [24]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)

(2538565823216, 2538565823360, 2538565823552)

In [25]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [26]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(2538565802176, 2538565802176, 2538552513408)

In [27]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

1. Using copy and deepcopy, we create three distinct Bus instances.
2. After bus1 drops 'Bill', he is also missing from bus2
3. Inspection of the passengers atributes shows that bus1 and bus2 share the same list object, because bus2 is a shallow copy of bus1.
4. bus3 is a deep copy of bus1, so its passengers attribute refers to another list.

The deepcopy function remembers the objects already copied to handle cyclic references gracefully.

In [28]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [29]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

## Function Parameters as References

Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become aliases of the actual arguments

The result of this scheme is that `a function may change any mutable object passed as a parameter`, but it cannot change the identity of those objects .

In [31]:
def f(a, b):
    a += b
    return a

x = 1
y = 2

In [32]:
f(x, y)

3

In [33]:
x, y

(1, 2)

In [34]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [35]:
a, b 

([1, 2, 3, 4], [3, 4])

In [36]:
t = (10, 20)
u = (30, 40)

f(t, u) 

(10, 20, 30, 40)

In [37]:
t, u

((10, 20), (30, 40))

1. The number x is unchanged.
2. The list a is changed.
3. The tuple t is unchanged.

### Mutable Types as Parameter Defaults: Bad Idea

In [38]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=[]) -> None:
        self.passengers = passengers
        
    def pick(self, name):
         self.passengers.append(name) 
    
    def drop(self, name):
        self.passengers.remove(name)

1. When the passengers argument is not passed, this parameter is bound to the default list object, which is initially empty.
2. This assignment makes self.passengers an alias for passengers, which is itself
an alias for the default list, when no passengers argument is given.
3. When the methods .remove() and .append() are used with self.passengers
we are actually mutating the default list, which is an attribute of the function object.

In [39]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [40]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [41]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [42]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [43]:
bus3.pick('Dave')
bus2.passengers 

['Carrie', 'Dave']

In [44]:
bus2.passengers is bus3.passengers

True

The problem is that Bus instances that don’t get an initial passenger list end up sharing the same passenger list among themselves.

In [45]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

### Defensive Programming with Mutable Parameters


1. Make a copy of the passengers list, or convert it to a list if it’s not one

In [46]:
class TwilightBus:
 """A bus model that makes passengers vanish"""
 def __init__(self, passengers=None):
    if passengers is None:
        self.passengers = [] 
    else:
        self.passengers = list(passengers) # 1
 
 def pick(self, name):
    self.passengers.append(name)
 
 def drop(self, name):
    self.passengers.remove(name) 


## del and Garbage Collection

The del statement deletes names, not objects. An object may be garbage collected as result of a del command, but only if the variable deleted holds the last reference to the object, or if the object becomes unreachable.Rebinding a variable may also cause the number of references to an object to reach zero, causing its destruction

In CPython, the primary algorithm for garbage collection is reference counting. Essentially, each object keeps count of how many references point to it. As soon as that refcount reaches zero, the object is immediately destroyed

In [49]:
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye(): 
    print('Gone with the wind...')
    
ender = weakref.finalize(s1, bye)
ender.alive

Gone with the wind...


True

In [50]:
del s1
ender.alive

True

In [51]:
s2 = 'spam' 

Gone with the wind...


In [52]:
ender.alive

False

1. s1 and s2 are aliases referring to the same set, {1, 2, 3}.
2. This function must not be a bound method of the object about to be destroyed or otherwise hold a reference to it.
3. Register the bye callback on the object referred by s1.
4. The .alive attribute is True before the finalize object is called.
5. As discussed, del does not delete an object, just a reference to it
6. Rebinding the last reference, s2, makes {1, 2, 3} unreachable. It is destroyed, the bye callback is invoked, and ender.alive becomes False

# Chapter Summary

Every Python object has an identity, a type, and a value. Only the value of an object changes over time.

If an immutable collection holds references to mutable items, then its value may actually change when the value of a mutable item changes.


- Augmented assignment with += or *= creates new objects if the lefthand variable is bound to an immutable object, but may modify a mutable object in place.
- Assigning a new value to an existing variable does not change the object previously bound to it. This is called a rebinding: the variable is now bound to a different object. If that variable was the last reference to the previous object, that object will be garbage collected.
- Function parameters are passed as aliases, which means the function may change any mutable object received as an argument. There is no way to prevent this, except making local copies or using immutable objects (e.g., passing a tuple instead of a list).
- Using mutable objects as default values for function parameters is dangerous because if the parameters are changed in place, then the default is changed, affecting every future call that relies on the default.